# Prototype of lipreading pipeline

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

#from torchtext.legacy.datasets import Multi30k
#from torchtext.legacy.data import Field, BucketIterator

#import spacy
import numpy as np

import random
import math
import time

# model file, encoder, decoder and seqtoseq
from model import *
# utils file
from utils import *
# Get landmark using vocadataset.py
from data.vocaset import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Get landmark from vocadaset class
trainset = vocadataset("train", landmark=True, mouthOnly=True)
landmark, labels = trainset[0]

In [2]:
trainset = vocadataset("train", landmark=True)
dataloader = DataLoader(trainset, batch_size=32, collate_fn=collate_fn)

In [3]:
vocabulary = vocabulary(blank='-', start='@', stop='#')

In [5]:
lan, lab = next(iter(dataloader))

In [4]:
# Create a mapping from characters to indices
vocabulary = vocabulary(blank='-', start='@', stop='#')
char_to_index = {char: index for index, char in enumerate(vocabulary)}
print(char_to_index)

{'-': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 27, '?': 28, ',': 29, '!': 30, '@': 31, '#': 32, ' ': 33}


In [21]:
lab[0]

'she had your dark suit in greasy wash water all year'

In [26]:
l = torch.tensor(lab)

ValueError: too many dimensions 'str'

In [25]:
labels = []
for ilab in lab:

    target_indices = [char_to_index[char] for char in [ilab]]
    labels.append(target_indices)

labels = torch.tensor(target_indices)

KeyError: 'she had your dark suit in greasy wash water all year'

In [ ]:
# Convert the sequence and target to indices
#sequence_indices = [char_to_index[char] for char in sequence]
target_indices = [char_to_index[char] for char in labels]
target_tensor = torch.tensor(target_indices)

In [ ]:

INPUT_DIM = landmark.shape[1]*3
HID_DIM = 128
output_dim = len(vocabulary)

enc = Encoder(INPUT_DIM, HID_DIM)
dec = Decoder(output_dim, HID_DIM)
model = Seq2Seq(enc, dec, 'cpu')

reshaped_landmark = torch.reshape(landmark, (landmark.shape[0], landmark.shape[1]*landmark.shape[2]))


In [ ]:
# adding start and stop character
start_landmark = torch.zeros(1, landmark.shape[1]*3)
stop_landmark = torch.ones(1, landmark.shape[1]*3)

final_landmarks = torch.cat((start_landmark, reshaped_landmark, stop_landmark), 0)

In [ ]:
# With batch

# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    for _, (landmarks, labels) in dataloader:
        # reshape the batch from [batch_size, frame_size, num_landmark, 3] to [batch_size, frame_size, num_landmark * 3] 
        landmark = torch.reshape(landmarks, (landmarks.shape[0], landmarks.shape[1]*landmarks.shape[2]))
        # label to char
        optimizer.zero_grad()
        output = model(reshaped_landmark, target_tensor[None,:])
        

        input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
        target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.long)
        
        loss = ctc_loss(output, target_tensor, input_lengths, target_lengths[0])
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
            #e = torch.argmax(output, dim=2).squeeze(1)
            #output_sequence = ''.join([vocabulary[index] for index in e])
            #print(output_sequence)
            torch.save(model.state_dict(), "models/model.pt")




In [ ]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    
    optimizer.zero_grad()
    output = model(reshaped_landmark, target_tensor[None,:])
    

    input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
    target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.long)
    
    loss = ctc_loss(output, target_tensor, input_lengths, target_lengths[0])
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        #e = torch.argmax(output, dim=2).squeeze(1)
        #output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)
        torch.save(model.state_dict(), "models/model.pt")




In [ ]:
# Decode the output sequence
#output_indices = torch.argmax(output, dim=2).squeeze(1)
#output_sequence = ''.join([vocabulary[index] for index in output_indices])

#print("Target Sequence:", labels.replace("@","").replace("#",""))
#print("Decoded Output:", process_string("-she had your dark suit in greasy  wwwaash  waterr  all-ll-  --yy--eeaa-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------"))